In [ ]:
%matplotlib tk
from Tkinter import TclError
import auto

In [ ]:
# Return a dict of the default parameters. Use kwargs to change a value, or set to None to exclude a parameter.
def params(I=-12, gleak=6.8, gKs=50, gKf=24.1, gNa=100, gNaP=0.8):
    p = {'I': I, 'gleak': gleak, 'gKs': gKs, 'gKf': gKf, 'gNa': gNa, 'gNaP': gNaP}
    for key, val in p.items():
        if val is None:
            del p[key]
    return p

params(I=None)

In [ ]:
# Integrate for stable equilibrium to start continuation from there
def integrate_to_equilibrium(**kwargs):
    integration = auto.run(e='drosophila', c='integration', **kwargs)
    return integration()[-1]  # last point

# Continue equilibrium forwards and backwards in I from init point
def continue_equilibrium(init, **kwargs):
    equilibrium = auto.run(init, c='equilibrium', **kwargs)
    equilibrium += auto.run(init, c='equilibrium', DS='-', **kwargs)
    equilibrium = auto.relabel(auto.merge(equilibrium))  # get single curve with unique labels
    return equilibrium

# Continue limit cycle in I from init point (for example a Hopf bifurcation)
def continue_limit_cycle(init, **kwargs):
    return auto.run(init, c='tube', **kwargs)

# Continue bifurcation point for changing second parameter (this has to be the second element in ICP, which has to be user defined)
def continue_in_2_params(init, second_param, second_param_uzstop=None, **kwargs):
    if 'ICP' not in kwargs:
        kwargs['ICP'] = ['I', second_param, 'PERIOD']
    if 'UZSTOP' not in kwargs:
        if second_param_uzstop is None:
            raise ValueError('Set either second_param_uzstop or UZSTOP directly')
        kwargs['UZSTOP'] = {'I': [-20, 100], second_param: second_param_uzstop}
        
    bifurcation_point = auto.run(init, c='2par', **kwargs)
    bifurcation_point += auto.run(init, c='2par', DS='-', **kwargs)
    bifurcation_point = auto.relabel(auto.merge(bifurcation_point))
    return bifurcation_point

def plot_bifurcation_diagram(points, **kwargs):
    try:
        auto.plot(points, 
          bifurcation_y=['V', 'MAX V', 'MIN V'], stability=True, use_labels=False, 
          grid=False, height=350, user_point_symbol='', xlabel='I / pA', ylabel='V / mV', color_list='blue blue green', **kwargs)
    except TclError:
        pass
    
def plot_2_params_diagram(points, second_param, **kwargs):
    try:
        auto.plot(points, bifurcation_x='I', bifurcation_y=second_param, use_labels=False,
                 grid=False, height=350, **kwargs)  # TODO: xlabel, ylabel, color_list
    except TclError:
        pass
    

Bifurcation diagram for default parameter values

In [ ]:
init = integrate_to_equilibrium()

In [ ]:
equilibrium = continue_equilibrium(init)

In [ ]:
#print equilibrium.summary()
plot_bifurcation_diagram(equilibrium)

In [ ]:
tube = continue_limit_cycle(equilibrium('HB1'))

In [ ]:
plot_bifurcation_diagram(equilibrium + tube)

2 parameter continuation for gleak

In [ ]:
# Continue saddle-node bifurcation (LP1) for changing gleak (2-parameter continuation)
saddle_node = continue_in_2_params(equilibrium('LP1'), 'gleak', [-10, 20], PAR=params(I=None, gleak=None))
plot_2_params_diagram(saddle_node, 'gleak')

In [ ]:
# Continue Hopf bifurcation (HB1) for changing gleak (2-parameter continuation)
hopf = continue_in_2_params(equilibrium('HB1'), 'gleak', 
                            UZSTOP={'I': [-20, 170], 'gleak': [-10, 20]}, PAR=params(I=None, gleak=None))
plot_2_params_diagram(hopf, 'gleak')

Bifurcation diagram for high gleak

In [ ]:
# As can be seen in the plot above, the system is governed by Hopf bifurcations for gleak >~ 8.6
# Make a bifurcation diagram for a high gleak value to see these bifurcations
new_gleak = 8.84

init2 = integrate_to_equilibrium(PAR=params(gleak=new_gleak))
equilibrium2 = continue_equilibrium(init, PAR=params(I=None, gleak=new_gleak), UZSTOP={'I': [-20, 170]})
tube2 = continue_limit_cycle(equilibrium('HB1'), PAR=params(gleak=new_gleak, I=None), 
                            UZSTOP={'I': [-20, 170], 'PERIOD': [0, 100]}, STOP=['LP2'], ILP=1)  # ILP=1 to detect folds

plot_bifurcation_diagram(equilibrium2 + tube2)

2 parameter continuation for gKs

In [ ]:
saddle_node = continue_in_2_params(equilibrium('LP1'), 'gKs', [0, 300], PAR=params(I=None, gKs=None))
plot_2_params_diagram(saddle_node, 'gKs')

In [ ]:
hopf = continue_in_2_params(equilibrium('HB1'), 'gKs', UZSTOP={'I': [-20, 300], 'gKs': [0, 1300]}, PAR=params(I=None, gKs=None))
plot_2_params_diagram(hopf, 'gKs')

In [ ]:
plot_2_params_diagram(saddle_node + hopf, 'gKs', minx=-20, maxx=120, miny=0, maxy=500)

2 parameter continuation for gKf

In [ ]:
saddle_node = continue_in_2_params(equilibrium('LP1'), 'gKf', [-10, 500], PAR=params(I=None, gKf=None))
saddle_node += continue_in_2_params(equilibrium('LP2'), 'gKf', [-10, 500], PAR=params(I=None, gKf=None))
plot_2_params_diagram(saddle_node, 'gKf')

In [ ]:
hopf = continue_in_2_params(equilibrium('HB1'), 'gKf', UZSTOP={'I': [-20, 170], 'gKf': [-10, 500]}, PAR=params(I=None, gKf=None))
plot_2_params_diagram(hopf, 'gKf')

2 parameter continuation for gNaP

In [ ]:
hopf = continue_in_2_params(equilibrium('HB1'), 'gNaP', UZSTOP={'I': [-100, 170], 'gKf': [0, 5]}, PAR=params(I=None, gNaP=None))
plot_2_params_diagram(hopf, 'gNaP')